In [1]:
import numpy as np
stuff=np.load("data.npz")
X_trn = stuff["X_trn"]
y_trn = stuff["y_trn"]
X_tst = stuff["X_tst"]

X_trn = np.float32(X_trn)
y_trn = np.float32(y_trn)
X_tst = np.float32(X_tst)



# Neural Networks

You will train several neural networks, each with a single hidden layer. These neural networks can be written as:

$$
f(x) = c + V \sigma(b + W x).
$$

Here:

- $x$ is the input, a vector of length $D$
- $W$ is a matrix of size $M \times D$ that maps input features to a hidden space
- $b$ is the bias term for the hidden layer, a vector of length $M$
- $\sigma(a)=\tanh(a)$ is the activation function. You will need that the derivative is $\frac{d\sigma(a)}{da} = 1-\tanh(a)^2$.
- $V$ is a matrix of size $O \times M$ that maps the hidden space to the output space
- $c$ is the bias term for the output space, a vector of length $O$

Note that $f(x) : \mathbb{R}^D\rightarrow\mathbb{R}^O$ is a function that maps a vector to a vector. We will refer to the $i$-the component of the output as $f(x)_i$.

For this problem, we will use the logistic loss, defined as

$$
L(y,f) = -f_y + \log \sum_{i=0}^3 \exp( f_i ),
$$

where $y \in \{0, 1, 2, 3\}$ is the *label* for the input $x$, and $f \in R^O$ is the output vector. Note that $f_y$ is therefore the $y^\text{th}$ component of the output vector $f$. Also be careful to note that here, we are indexing $f$ from 0 instead of 1.






 




**Question 1** (5 points) Write a function to evaluate the neural network and loss. Your function should have the following signature:

```python
def prediction_loss(x,y,W,V,b,c):
    # do stuff here
    return L
```

This should return a scalar. Give your function directly in your report.

In [2]:
import numpy as np

def prediction_loss(W,V,b,c,x,y):
    f = c + V @ np.tanh(b + W @ x)

    log_term  = np.log(np.sum(np.exp(f))) 
    L = -f[y] + log_term
    return L


**Question 2** (10 points) Write a function to evaluate the gradient of the neural network. Your function should have the following signature. Do not use any packages outside of numpy.

```python
def prediction_grad(x,y,W,V,b,c):
    # do stuff here
    return dLdW, dLdV, dLdb, dLdc
```

Each returned array should be the same size as the input, and contain the corresponding gradient. So, for example,`dLdW` is the derivatives $\nabla_W L(y,f(x))$. Give your function directly in your report.

In [3]:
def prediction_grad(x, y, W, V, b, c):
    # Forward pass
    h = b + W @ x                  # hidden layer pre-activation
    sigma_h = np.tanh(h)          # hidden layer activation
    f = c + V @ sigma_h           # output
    
    # Compute softmax probabilities
    exp_f = np.exp(f)
    sum_exp_f = np.sum(exp_f)
    probs = exp_f / sum_exp_f
    
    # Backward pass
    # Start with df/df = probs - one_hot(y)
    df = probs.copy()
    df[y] -= 1
    
    # dL/dV = df/df * df/dV = df * sigma_h^T
    dLdV = np.outer(df, sigma_h)
    
    # dL/dc = df/df * df/dc = df * 1
    dLdc = df
    
    # dL/d(sigma_h) = df/df * df/d(sigma_h) = V^T * df
    d_sigma_h = V.T @ df
    
    # dL/dh = dL/d(sigma_h) * d(sigma_h)/dh = d_sigma_h * (1 - tanh^2(h))
    dh = d_sigma_h * (1 - sigma_h**2)
    
    # dL/dW = dL/dh * dh/dW = dh * x^T
    dLdW = np.outer(dh, x)
    
    # dL/db = dL/dh * dh/db = dh * 1
    dLdb = dh
    
    return dLdW, dLdV, dLdb, dLdc


**Question 3** (10 points) Take the following inputs, where there are 3 hidden units and 2 outputs ($y = 0 \text{ or } y = 1$):

$$
\begin{aligned}
x &= [1,2]\\
y &= 1 \\
W &= \begin{pmatrix}0.5 & -1 \\ -0.5 & 1 \\ 1 & .5\end{pmatrix}\\
V &= \begin{pmatrix}-1 & -1 & 1 \\ 1 & 1 & 1  \end{pmatrix}\\
b &= [0,0,0]\\
c &= [0,0]
\end{aligned}
$$

Run the function from the previous question to compute the gradient with respect to $W$, $V$, $b$, and $c$. Give the results directly in your report, organized as you see above.


In [4]:
x = np.array([1,2])
y = 1
W = np.array([[0.5,-1], [-0.5,1], [1,0.5]])
V = np.array([[-1,-1,1],[1,1,1]])
b = np.array([0,0,0])
c = np.array([0,0])

results = prediction_grad(x,y,W,V,b,c)

terms = ['W','V','b','c']

for result, term in zip(results, terms):
    print(f'\n Grad {term}: {result}')


 Grad W: [[-0.18070664 -0.36141328]
 [-0.18070664 -0.36141328]
 [ 0.          0.        ]]

 Grad V: [[-0.45257413  0.45257413  0.48201379]
 [ 0.45257413 -0.45257413 -0.48201379]]

 Grad b: [-0.18070664 -0.18070664  0.        ]

 Grad c: [ 0.5 -0.5]




## JAX

The next several questions use the `JAX` toolbox, which you can install via `pip install jax jaxlib`. The documentation for using grad from `JAX` can be found on this page:

https://jax.readthedocs.io/en/latest/quickstart.html#taking-derivatives-with-jax-grad


**Question 4** (5 points) Write a function to evaluate the same gradient as in Question 2 using the Jax toolbox (Hint: You will need to import the NumPy wrapper, `import jax.numpy as np`, and the `grad` high-order function, `from jax import grad`).

```python
def prediction_grad_jax(x,y,W,V,b,c):
    # do stuff here
    return dLdW, dLdV, dLdb, dLdc
```

Give your function directly in your report. (You do not need to give any outputs from your function, but it is suggested to check the results against Q3 since if they are different, one must be wrong!)

*Hint*: For JAX to work, the returned value from the loss function should be a scalar, you can use squeeze to ensure that. (e.g.  `L = L.squeeze()`)

In [5]:
import jax.numpy as jnp
from jax import grad


def prediction_grad_jax(W, V, b, c, x, y):
    # Create the gradient function
    grad_fn = grad(prediction_loss_jax, argnums=(0, 1, 2, 3))
    # Call the gradient function with your inputs
    return grad_fn(W, V, b, c, x, y)


def prediction_loss_jax(W,V,b,c,x,y):
    f = c + V @ jnp.tanh(b + W @ x)

    log_term  = jnp.log(jnp.sum(jnp.exp(f))) 
    L = -f[y] + log_term
    return L

In [6]:

x = np.array([1.0,2.0], dtype=np.float32)
y = np.int32(1)

W = np.array([[0.5,-1.0], 
              [-0.5,1.0], 
              [1.0,0.5]], dtype=np.float32)

V = np.array([[-1.0,-1.0,1.0],

              [1.0,1.0,1.0]], dtype=np.float32)
b = np.array([0.0,0.0,0.0], dtype=np.float32)
c = np.array([0.0,0.0], dtype=np.float32)

grads = (prediction_grad_jax(W,V,b,c,x,y))

dLdW, dLdV, dLdb, dLdc = prediction_grad_jax(W, V, b, c, x, y)
print(f'Grad W: {dLdW}')
print(f'Grad V: {dLdV}')
print(f'Grad b: {dLdb}')
print(f'Grad c: {dLdc}')

Grad W: [[-0.18070672 -0.36141345]
 [-0.18070672 -0.36141345]
 [ 0.          0.        ]]
Grad V: [[-0.4525741  0.4525741  0.4820138]
 [ 0.4525741 -0.4525741 -0.4820138]]
Grad b: [-0.18070672 -0.18070672  0.        ]
Grad c: [ 0.5 -0.5]



**Question 5** (5 points) Update your function from Question 1. Instead of taking a single input x and a single output y, take an 2D of inputs X (where the first dimension indexes the different examples) and a 1D array of outputs Y. Also, take a regularization constant `λ` and apply squared regularization to `W` and `V`. Do not regularize `b` or `c`. Your function should be the sum of the logistic losses for each example in the dataset, plus the regularizer loss applied to `W` and `V`. (To be explicit, the regularizer could be written as $\lambda \left( \sum_{vm} W_{vm}^2 + \sum_{mi} V_{mi}^2 \right)$.) Give your function directly in your report.

```python
def prediction_loss_full(X,Y,W,V,b,c,λ):
    # do stuff here
    return L # include regularization
```

In [7]:
import jax.numpy as jnp
from jax import grad

def prediction_loss_full(X, Y, W, V, b, c, lam):
    total_loss = 0.0
    
    # Loop through each example
    for i in range(len(Y)):
        # Forward pass
        f = c + V @ jnp.tanh(b + W @ X[i])
        
        y_idx = jnp.asarray(Y[i], dtype=jnp.int32)
        total_loss += -f[y_idx] + jnp.log(jnp.sum(jnp.exp(f)))  # Cast Y[i] to int for indexing
    
    # Add regularization
    total_loss += lam * (jnp.sum(W**2) + jnp.sum(V**2))
    
    return total_loss


**Question 6** (5 points) ****Update your gradient function to work on a full dataset and include regularization, as in the previous question. Again, you should use JAX. Give your function directly in your report.

```python
def prediction_grad_full(X,Y,W,V,b,c,λ):
    # do stuff here
    return dLdW, dLdV, dLdb, dLdc
```

In [8]:

def prediction_grad_full(X,Y,W,V,b,c,lam):
    # Get gradients for all parameters using argnums
    grads = grad(prediction_loss_full, argnums=(0, 1, 2, 3))(X,Y,W,V,b,c,lam)
    return grads


**Question 7** (15 points) Here is psuedo-code to optimize a function $h(w)$ by gradient descent with momentum.

```python
avg_grad = 0
for iter = 1, 2, ... max_iters:
    avg_grad = (1 - momentum) * avg_grad + momentum * ∇h(w)
    w = w - stepsize * avg_grad
```

For each size of the hidden layer, $M \in \{5, 40, 70\}$, train your neural network on the main data for this homework. Weights for layers $W$ and $V$ should be initialized by sampling from $\frac{\mathcal N(0, 1)}{\sqrt{D}}$, where $\mathcal N(0, 1)$ is the standard normal distribution and $D$ is the number of input dimensions for that layer. Weight for $b$ and $c$ should be initialized as zeros. 

Use gradient descent with momentum, with 1000 iterations, a step size of .000025, a momentum of 0.1, and $\lambda = 1$.

Report your code directly in the report, and the following:

1. For each value of $M$, what is the total training time (in ms) for all iterations. (Give a table with 3 entries.)
2. Make a plot of the training objective (regularized loss) as a function of iterations. This should be a single plot with 3 curves, one for each value of $M$. Include the plot in your report.

In [17]:
import time
import jax.numpy as jnp
from jax import random
import matplotlib.pyplot as plt

M_values = [3] 
D = X_trn.shape[1]
O = y_trn.shape[0]

avg_grad = 0
step_size = 0.000025
momentum = 0.1
lam = 1
max_iters = 100
key = random.PRNGKey(0)
training_losses = {M: [] for M in M_values}

for M in M_values:
    W = random.normal(key, shape=(M, D))
    V = random.normal(key, shape=(O, M))
    b = jnp.zeros([M, 1]) 
    c = jnp.zeros([O, 1])
    start_time = time.perf_counter()

    for iter in range(max_iters):
        dLdW, dLdV, dLdb, dLdc = prediction_grad_full(X_trn, y_trn, W, V, b, c, lam)
        avg_grad = (1 - momentum) * avg_grad + momentum * dLdW
        W -= step_size * avg_grad
        loss = prediction_loss_full(X_trn, y_trn, W, V, b, c, lam)
        training_losses[M].append(float(loss))
    
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(f"Elapsed time for M={M}: {elapsed_time:.2f} seconds")

# plot training losses
plt.figure(figsize=(10, 6))
for M in M_values:
    plt.plot(training_losses[M], label=f'M={M}')
plt.xlabel('Iterations')
plt.ylabel('Training Loss')
plt.title('Training Loss vs Iterations for Different M')
plt.legend()
plt.grid()
plt.show()

Training network with M=5


TypeError: dot_general requires contracting dimensions to have the same shape, got (841,) and (3,).



**Question 8** (10 points) Make a single train-validation split of the data with 50% used for training and 50% for testing. Train your neural network using the parameters above for each value of $M$ and give the estimated generalization error. Again, using the same initial weights generated using the scheme above. Then, retrain your network on *all* the data,  make predictions for the test data, and report your prediction for the test data (include in the `report_src/` folder). 

For the test data predictions, you will be required to submit a `.csv` file with two columns: an "Id" column and a "Category" column classifying the integer prediction for each element in `X_tst`. A sample solution using randomly predicted outputs can be generated as follows:

```python
import numpy as np
import csv

def write_csv(y_pred, filename):
    """Write a 1d numpy array to a Kaggle-compatible .csv file"""
    with open(filename, 'w') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Id', 'Category'])
        for idx, y in enumerate(y_pred):
            csv_writer.writerow([idx, y])

data = np.load('data.npz')
X_tst = data['X_tst']
y_pred = np.random.randint(0, 3, size=len(X_tst)) # random predictions
write_csv(y_pred, 'submission.csv')
```

You can use the write_csv helper function in your code if you find it helpful to ensure that your solution is in the correct format.

Report your code directly in the report, and the following:

- What value of $M$ you chose.
- The estimated generalization error.
- The prediction on the test data (in a CSV file, named `submission.csv`).